In [1]:
import os
from bs4 import BeautifulSoup
from time import sleep
from pymongo import MongoClient
from pprint import pprint
import concurrent.futures
import random
from copy import deepcopy
from datetime import datetime
import time
import requests
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
#helper files
import get_proxies as proxies
from extractors import get_data 

#### Pull listing_url documents from mongoDB

In [4]:
db = MongoClient().get_database('philadelphia_home_prices')

In [5]:
col = db.get_collection('listing_urls')

In [6]:
cursor = db.listing_urls.find({})
listings = list(cursor)

In [7]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [8]:
#check first 5 in list
listing_urls_list[0:5]

['https://www.realtor.com/realestateandhomes-detail/1232-N-Mascher-St-Apt-A_Philadelphia_PA_19122_M49182-95881',
 'https://www.realtor.com/realestateandhomes-detail/212-Carpenter-St-Rear-C_Philadelphia_PA_19147_M34206-29412',
 'https://www.realtor.com/realestateandhomes-detail/1211-E-Cheltenham-Ave_Philadelphia_PA_19124_M33700-20336',
 'https://www.realtor.com/realestateandhomes-detail/1002-Brandywine-St_Philadelphia_PA_19123_M39147-93479',
 'https://www.realtor.com/realestateandhomes-detail/2607-N-31st-St_Philadelphia_PA_19132_M32883-74202']

#### New collection for listing data

In [9]:
db.list_collection_names()

['prices_messy', 'listing_data', 'listing_urls']

In [10]:
#uncomment the code you need to use
db.drop_collection('listing_data')
db.create_collection('listing_data')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'philadelphia_home_prices'), 'listing_data')

In [11]:
col = db.get_collection('listing_data')

In [12]:
col.count_documents({})

0

#### Get free proxies from https://free-proxy-list.net/

In [13]:
#call function to get list of proxies
proxies_address = proxies.proxy_list()

2020-11-07 17:17:02,757 root   DEBUG    === Initialized Proxy Parsers ===
2020-11-07 17:17:02,757 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2020-11-07 17:17:02,758 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2020-11-07 17:17:02,758 root   DEBUG    =================================
2020-11-07 17:17:03,228 root   DEBUG    Added 300 proxies from FreeProxy
2020-11-07 17:17:03,955 http_request_randomizer.requests.parsers.PremProxyParser DEBUG    Pages: {'', '09.htm', '08.htm', '06.htm', '02.htm', '05.htm', '04.htm', '07.htm', '03.htm'}
2020-11-07 17:17:04,509 http_request_randomizer.requests.parsers.js.UnPacker INFO     JS UnPacker init path: https://premproxy.com/js/32dbd.js
2020-11-07 17:17:05,073 http_request_randomizer.requests.parsers.js.UnPacker DEBUG    portmap: {'rd034': '8080', 'ra998': '32108', 'r38c9': '8087', 'r57e1': '8088', 'r06e8': '999', 'r85dc': '3128', 'rc

In [14]:
len(proxies_address)

615

#### Pick which urls in list by index

In [15]:
#might not be needed
scraped_urls = []
bad_proxies = []
bad_urls = []

In [16]:
#only run once
remaining_listing_urls_list = deepcopy(listing_urls_list)

In [19]:
def scrape_requests(url):
    listing_data = {}
    try:
        PROXY = random.choice(proxies_address)
        proxies = {'http': PROXY,
                   'https': PROXY,
                  }

        headers = {
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
                    "Accept-Encoding": "gzip, deflate, sdch, br",
                    "Accept-Language": "en-US,en;q=0.8",
                    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36",
                    }

       
        try: 
            response = requests.get(url,headers=headers,proxies=proxies)
        except:
            response = False
            
        soup = BeautifulSoup(response.text, 'html.parser')
        listing_data = get_data(soup, url)

        
        if response.status_code == 200 and listing_data['Address'] != np.nan:
            scraped_urls.append(url)
            remaining_listing_urls_list.remove(url)
            col.insert_one(listing_data)
            print(listing_data)
    
    except:
        if response == False:
            bad_proxies.append(PROXY)
            proxies_address.remove(PROXY)
            print('no response, bad proxy')
        elif response.status_code != 200:
            bad_proxies.append(PROXY)
            proxies_address.remove(PROXY)
            print('not 200, bad proxy')
        else:
            bad_urls.append(url)
            remaining_listing_urls_list.remove(url)
    
    print('______________________________________________________')
    print(f'   Remaining urls: {len(remaining_listing_urls_list)}')
    print(f'     Scraped urls: {len(scraped_urls)}')
    print(f'Good proxies left: {len(proxies_address)}')
    print(f'      Bad proxies: {len(bad_proxies)}')
    
    
    if len(proxies_address) < 100:
        sleep(random.uniform(60,120))
    
    return listing_data

In [20]:
def threaded_scrape(remaining_listing_urls_list):
    threads = min(50, len(proxies_address))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(scrape_requests, remaining_listing_urls_list)

In [ ]:
threaded_scrape(remaining_listing_urls_list)

no response, bad proxy
______________________________________________________
   Remaining urls: 1100
     Scraped urls: 0
Good proxies left: 614
      Bad proxies: 1
no response, bad proxy
______________________________________________________
   Remaining urls: 1100
     Scraped urls: 0
Good proxies left: 613
      Bad proxies: 2
no response, bad proxy
______________________________________________________
   Remaining urls: 1100
     Scraped urls: 0
Good proxies left: 612
      Bad proxies: 3
no response, bad proxy
______________________________________________________
   Remaining urls: 1100
     Scraped urls: 0
Good proxies left: 611
      Bad proxies: 4
no response, bad proxy
______________________________________________________
   Remaining urls: 1100
     Scraped urls: 0
Good proxies left: 610
      Bad proxies: 5
no response, bad proxy
______________________________________________________
   Remaining urls: 1100
     Scraped urls: 0
Good proxies left: 609
      Bad proxies: 

no response, bad proxy      Bad proxies: 25


______________________________________________________
   Remaining urls: 1094
     Scraped urls: 6
Good proxies left: 590
      Bad proxies: 25
     Scraped urls: 6
Good proxies left: 590{'Address': '6128 N 7th St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19120', 'Price': 161000, 'Days on Market': 50, 'SQFT House': 2012, 'SQFT Lot': 1440, 'Full Baths': 2, 'Half Baths': 0, 'Bedrooms': 4, 'Garage': [' 1'], 'Master Bath': 'No', 'Cooling': 'No', 'Heating': ['Radiator', 'Yes', 'Unknown'], 'Annual Tax': 1922, 'House Type': [' Interior Row/Townhouse', ' Multi family'], 'Build Year': 1945, 'Remodel Year': 2020, 'Neighborhood': 'Olney', 'Neighborhood median sales': 123000, 'Neighborhood Median DOM': 67, 'Neighborhood Price SQFT': 113, 'Association': 'No', 'Association Monthly': 0, 'URL': 'https://www.realtor.com/realestateandhomes-detail/6128-N-7th-St_Philadelphia_PA_19120_M30359-01965', 'crawl time': datetime.datetime(2020, 11

      Bad proxies: 31{'Address': '310 S Front St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19106', 'Price': 1850000, 'Days on Market': 65, 'SQFT House': 3523, 'SQFT Lot': 81984, 'Full Baths': 3, 'Half Baths': 1, 'Bedrooms': 4, 'Garage': [' 3'], 'Master Bath': 'No', 'Cooling': ['Central A/C, Zoned Cooling, Programmable Thermostat Cooling, Energy Star Cooling System'], 'Heating': ['Forced Air Heating, Zoned Heating, Programmable Thermostat Heating', 'Yes'], 'Annual Tax': 15849, 'House Type': [' Interior Row/Townhouse', ' Condo'], 'Build Year': 1971, 'Remodel Year': 2020, 'Neighborhood': 'Not Found', 'Neighborhood median sales': 250000, 'Neighborhood Median DOM': 63, 'Neighborhood Price SQFT': 195, 'Association': 'Yes', 'Association Monthly': 616, 'URL': 'https://www.realtor.com/realestateandhomes-detail/310-S-Front-St_Philadelphia_PA_19106_M45281-45249', 'crawl time': datetime.datetime(2020, 11, 7, 17, 19, 32, 48635), '_id': ObjectId('5fa73994653eadd27d60c062')}
____

no response, bad proxy
______________________________________________________
   Remaining urls: 1080
     Scraped urls: 20
Good proxies left: 555
      Bad proxies: 61
no response, bad proxyno response, bad proxyno response, bad proxy{'Address': '3702 Calumet St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19129', 'Price': 349000, 'Days on Market': 39, 'SQFT House': 1632, 'SQFT Lot': 1178, 'Full Baths': 2, 'Half Baths': 0, 'Bedrooms': 4, 'Garage': 'No', 'Master Bath': 'No', 'Cooling': ['Central A/C', 'Central'], 'Heating': ['Hot Water', 'Yes', 'Hot Water'], 'Annual Tax': 3428, 'House Type': [' End of Row/Townhouse', ' Multi family'], 'Build Year': 1939, 'Remodel Year': 2020, 'Neighborhood': 'East Falls', 'Neighborhood median sales': 317500, 'Neighborhood Median DOM': 56, 'Neighborhood Price SQFT': 220, 'Association': 'No', 'Association Monthly': 0, 'URL': 'https://www.realtor.com/realestateandhomes-detail/3702-Calumet-St_Philadelphia_PA_19129_M35469-22314', 'crawl ti

______________________________________________________no response, bad proxy   Remaining urls: 1074
     Scraped urls: 26
Good proxies left: 537
      Bad proxies: 80


______________________________________________________
{'Address': '52104 Delaire Landing Rd Unit 104,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19114', 'Price': 225000, 'Days on Market': 57, 'SQFT House': 992, 'SQFT Lot': 992, 'Full Baths': 1, 'Half Baths': 0, 'Bedrooms': 2, 'Garage': 'No', 'Master Bath': 'No', 'Cooling': ['Central A/C'], 'Heating': ['Forced Air Heating', 'Yes'], 'Annual Tax': 1708, 'House Type': [' Unit/Flat/Apartment', ' Condo'], 'Build Year': 1972, 'Remodel Year': 2020, 'Neighborhood': 'Not Found', 'Neighborhood median sales': 250000, 'Neighborhood Median DOM': 63, 'Neighborhood Price SQFT': 195, 'Association': 'No', 'Association Monthly': 322, 'URL': 'https://www.realtor.com/realestateandhomes-detail/52104-Delaire-Landing-Rd-104_Philadelphia_PA_19114_M43067-69808', 'crawl time':

...

Good proxies left: 417
      Bad proxies: 208
      Bad proxies: 208no response, bad proxy


______________________________________________________     Scraped urls: 75no response, bad proxy
no response, bad proxy

   Remaining urls: 1024____________________________________________________________________________________________________________

   Remaining urls: 1024

     Scraped urls: 76
   Remaining urls: 1024
     Scraped urls: 76
Good proxies left: 414
      Bad proxies: 211
Good proxies left: 414     Scraped urls: 76

      Bad proxies: 211{'Address': '4618 Pennypack St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19136', 'Price': 208000, 'Days on Market': 59, 'SQFT House': 1696, 'SQFT Lot': 2409, 'Full Baths': 1, 'Half Baths': 1, 'Bedrooms': 3, 'Garage': [' 1'], 'Master Bath': 'No', 'Cooling': 'No', 'Heating': 'No', 'Annual Tax': 2160, 'House Type': [' Multi family'], 'Build Year': 1962, 'Remodel Year': 2020, 'Neighborhood': 'North Delaware', 'Neighborhood me

______________________________________________________{'Address': '251 Dimarco Dr,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19154', 'Price': 268000, 'Days on Market': 38, 'SQFT House': 1056, 'SQFT Lot': 5444, 'Full Baths': 1, 'Half Baths': 1, 'Bedrooms': 3, 'Garage': 'No', 'Master Bath': 'No', 'Cooling': ['Central A/C', 'Central'], 'Heating': ['Forced Air Heating', 'Yes', 'Unknown'], 'Annual Tax': 3063, 'House Type': [' Twin/Semi-Detached', ' Single family'], 'Build Year': 1955, 'Remodel Year': 2020, 'Neighborhood': 'Millbrook', 'Neighborhood median sales': 260000, 'Neighborhood Median DOM': 62, 'Neighborhood Price SQFT': 199, 'Association': 'No', 'Association Monthly': 0, 'URL': 'https://www.realtor.com/realestateandhomes-detail/251-Dimarco-Dr_Philadelphia_PA_19154_M34575-43360', 'crawl time': datetime.datetime(2020, 11, 7, 17, 24, 48, 105165), '_id': ObjectId('5fa73ad0653eadd27d60c0a6')}

   Remaining urls: 1017
no response, bad proxy

      Bad proxies: 223
____


Good proxies left: 391no response, bad proxy
______________________________________________________no response, bad proxy

   Remaining urls: 1010
     Scraped urls: 90
______________________________________________________
   Remaining urls: 1010
     Scraped urls: 90      Bad proxies: 236


Good proxies left: 389Good proxies left: 389
no response, bad proxy

______________________________________________________
   Remaining urls: 1009
no response, bad proxy
______________________________________________________
   Remaining urls: 1009
{'Address': '2522 Ridge Ave,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19121', 'Price': 175000, 'Days on Market': 128, 'SQFT House': 1998, 'SQFT Lot': 1192, 'Full Baths': 2, 'Half Baths': 0, 'Bedrooms': 6, 'Garage': 'No', 'Master Bath': 'No', 'Cooling': 'No', 'Heating': ['Forced Air Heating', 'Yes'], 'Annual Tax': 1230, 'House Type': [' Interior Row/Townhouse', ' Commercial'], 'Build Year': 1915, 'Remodel Year': 2020, 'Neighborhood

{'Address': '748 N Holly St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19104', 'Price': 136000, 'Days on Market': 17, 'SQFT House': 1450, 'SQFT Lot': 1488, 'Full Baths': 2, 'Half Baths': 0, 'Bedrooms': 3, 'Garage': 'No', 'Master Bath': 'No', 'Cooling': ['Ceiling Fan'], 'Heating': ['Radiant', 'Yes', 'Unknown'], 'Annual Tax': 748, 'House Type': [' Interior Row/Townhouse', ' Multi family'], 'Build Year': 1925, 'Remodel Year': 2020, 'Neighborhood': 'Belmont', 'Neighborhood median sales': 45000, 'Neighborhood Median DOM': 45, 'Neighborhood Price SQFT': 140, 'Association': 'No', 'Association Monthly': 0, 'URL': 'https://www.realtor.com/realestateandhomes-detail/748-N-Holly-St_Philadelphia_PA_19104_M34509-03878', 'crawl time': datetime.datetime(2020, 11, 7, 17, 25, 56, 522379), '_id': ObjectId('5fa73b14653eadd27d60c0b6')}
______________________________________________________
   Remaining urls: 1000
     Scraped urls: 100
Good proxies left: 372
      Bad proxies: 254
{'Add

no response, bad proxy
______________________________________________________
   Remaining urls: 996
     Scraped urls: 104
Good proxies left: 349
      Bad proxies: 280
no response, bad proxy
______________________________________________________
   Remaining urls: 996
     Scraped urls: 104
Good proxies left: 348
      Bad proxies: 281


### Sandbox

In [ ]:
count = 0
while count < 20:
    listing_data = scrape(remaining_listing_urls_list)
    #print(count)
    #print(listing_data)
    count += 1

In [ ]:
"85.238.167.170:51915" in proxies_address

In [ ]:
col.count_documents({})

In [ ]:
'https://www.realtor.com/realestateandhomes-detail/4521-Tolbut-St_Philadelphia_PA_19136_M45152-02595' in remaining_listing_urls_list

In [ ]:
len(remaining_listing_urls_list)

In [ ]:
len(listing_urls_list)

In [ ]:
t0 = datetime.now()
sleep(1)
t1 = datetime.now()
(t1-t0) 

In [ ]:
t0 = time.time()
sleep(1)
t1 = time.time()
t1-t0

In [ ]:
type(t0)

In [ ]:
time.time()

In [ ]:
driver

In [ ]:
proxies_address[:10]

In [ ]:
time.time()

In [ ]:
response = requests.get(url,headers=headers,proxies=proxies)

In [ ]:
t1 = time.time()

In [ ]:
listing_data

In [ ]:
time.time()